# Factor Investing and Fama-French model

**Copyright &copy; 2020 Ondrej Martinsky, All rights reserved**

[www.github.com/omartinsky/FamaFrench](https://github.com/omartinsky/FamaFrench)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wx
from ipywidgets import interact
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from pandas.plotting import register_matplotlib_converters
from IPython.core.display import display, HTML
import famafrench
register_matplotlib_converters()

c:\users\ondrej\python\wpy64-3800\python-3.8.0.amd64\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def create_ma_radios(value):
    return wx.RadioButtons(value=value, options=famafrench.lookup_ma.keys(), description='Roll Avg')

# Fama French

$ r = R_f + b_m \cdot \text{MER} + b_s \cdot \text{SMB} + b_v \cdot \text{HML} + b_p \cdot \text{RMW} + b_i \cdot \text{CMA} $

$r$ is expected portfolio return \
$b_m$, $b_s$, $b_v$, $b_p$, $b_i$ are factor sensitivities

# Factors Analysis
$\text{MER}$ is excess return of a broad market over the risk-free rate ($R_m - R_f$) (**market factor**) \
$\text{SMB}$ is the average return on the *small* stock portfolios minus *big* stock portfolios (**size factor**) \
$\text{HML}$ is the average return on *high* book value portfolios minus *low* book value (**value factor**)  \
$\text{RMW}$ is the average return on *robust* minus *weak* operating profitability portfolios (**profitability factor**) \
$\text{CMA}$ Conservative minus aggresive investments, proxied by the y/y growth in company asset value (**investment factor**)

Definitions: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

In [3]:
display(HTML('<b>Factors time series</b>'), famafrench.df_factors)
display(HTML('<b>Factors Statistics</b>'), famafrench.df_factors.describe().T)
display(HTML('<b>Factors Correlation Matrix</b>'), famafrench.df_factors.corr())

,MER,SMB,HML,RMW,CMA,RF
1963-07-01,-0.0067,0.0000,-0.0032,-0.0001,0.0015,0.00012
1963-07-02,0.0079,-0.0027,0.0027,-0.0007,-0.0019,0.00012
1963-07-03,0.0063,-0.0017,-0.0009,0.0017,-0.0033,0.00012
1963-07-05,0.0040,0.0008,-0.0028,0.0008,-0.0033,0.00012
1963-07-08,-0.0063,0.0004,-0.0018,-0.0029,0.0013,0.00012
...,...,...,...,...,...,...
2020-04-24,0.0144,0.0029,-0.0009,-0.0014,0.0016,0.00000
2020-04-27,0.0173,0.0220,0.0274,-0.0012,0.0020,0.00000
2020-04-28,-0.0044,0.0165,0.0289,0.0139,0.0074,0.00000
2020-04-29,0.0292,0.0227,0.0204,0.0122,-0.0081,0.00000


,count,mean,std,min,25%,50%,75%,max
MER,14306.0,0.000254,0.010113,-0.1744,-0.00410,0.00050,0.00490,0.11350
SMB,14306.0,0.000066,0.005257,-0.1117,-0.00270,0.00020,0.00300,0.06080
HML,14306.0,0.000132,0.005160,-0.0472,-0.00230,0.00010,0.00240,0.04830
RMW,14306.0,0.000127,0.003680,-0.0292,-0.00170,0.00010,0.00180,0.04400
CMA,14306.0,0.000129,0.003643,-0.0594,-0.00170,0.00010,0.00190,0.02530
RF,14306.0,0.000179,0.000125,0.0000,0.00008,0.00018,0.00024,0.00061


,MER,SMB,HML,RMW,CMA,RF
MER,1.000000,-0.098804,-0.170869,-0.199234,-0.364157,-0.018872
SMB,-0.098804,1.000000,0.093717,-0.282457,0.017021,-0.011284
HML,-0.170869,0.093717,1.000000,-0.048140,0.541814,0.026391
RMW,-0.199234,-0.282457,-0.048140,1.000000,0.063369,0.002674
CMA,-0.364157,0.017021,0.541814,0.063369,1.000000,0.023622
RF,-0.018872,-0.011284,0.026391,0.002674,0.023622,1.000000


In [4]:
display(HTML("<b>Distribution Analysis</b>"))
    
wx_ff=wx.Dropdown(options=famafrench.lookup_factors.keys(), description='Factor')
wx_ma=create_ma_radios('1D')
wx_out1=wx.interactive_output(famafrench.plot_factor_hists, {'ff':wx_ff, 'ma':wx_ma})
wx_out2=wx.interactive_output(famafrench.plot_factor_timeseries, {'ff':wx_ff, 'ma':wx_ma})
display(wx.HBox([wx_ff, wx_ma]), wx_out1, wx_out2)

Output()

Output()

# Portfolio Analysis

In [5]:
def create_factor_radios():
    return {
        'f_mer': wx.Checkbox(description='MER', indent=False, layout=chkbox_layout, value=True),
        'f_smb': wx.Checkbox(description='SMB', indent=False, layout=chkbox_layout, value=True),
        'f_hml': wx.Checkbox(description='HML', indent=False, layout=chkbox_layout, value=True),
        'f_rmw': wx.Checkbox(description='RMW', indent=False, layout=chkbox_layout),
        'f_cma': wx.Checkbox(description='CMA', indent=False, layout=chkbox_layout),
    }

def create_portfolio_dropdown():
    return wx.Dropdown(options=famafrench.portfolios.keys(), description='Portfolio', layout=wx.Layout(width='500px'))


chkbox_layout = wx.Layout(width='70px')
wx_pname = create_portfolio_dropdown()
wx_ma = create_ma_radios('3Y')
wx_factor_list = create_factor_radios()
wx_daterange=wx.SelectionRangeSlider(options=range(1960,2021), index=(0, len(range(1960,2021))-1), description='Range', continuous_update=False, layout=wx.Layout(width='500px'))
wx_out=wx.interactive_output(famafrench.fit_portfolio_returns, {'pname':wx_pname, **wx_factor_list , 'ma': wx_ma, 'daterange': wx_daterange})
wx_out.layout.height='700px'
wx_out_r2hist = wx.interactive_output(famafrench.draw_R2_hist, {'pname':wx_pname, 'daterange': wx_daterange})
wx_out_r2series = wx.interactive_output(famafrench.draw_R2_series, {'pname':wx_pname, **wx_factor_list, 'daterange': wx_daterange})
display(
    wx_pname, 
    wx.HBox([wx_daterange, *(wx_factor_list.values())]),
    wx.HBox([wx_out_r2hist, wx_out_r2series]),
    wx.HBox([wx_out, wx_ma]))

Dropdown(description='Portfolio', layout=Layout(width='500px'), options=('1. Bivariate SMB+HML (2x3) ➤ Value W…